##### Copyright 2019 The TensorFlow Authors.

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

_Note: This notebook is taken from [this official tutorial](https://www.tensorflow.org/text/tutorials/text_generation) and refactored to run on the Tensorflow version used in class._

# Text generation with an RNN

This tutorial demonstrates how to generate text using a character-based RNN. You will work with a dataset of Shakespeare's writing from Andrej Karpathy's [The Unreasonable Effectiveness of Recurrent Neural Networks](http://karpathy.github.io/2015/05/21/rnn-effectiveness/). Given a sequence of characters from this data ("Shakespear"), train a model to predict the next character in the sequence ("e"). Longer sequences of text can be generated by calling the model repeatedly.

Note: Enable GPU acceleration to execute this notebook faster. In Colab: *Runtime > Change runtime type > Hardware accelerator > GPU*.

This tutorial includes runnable code implemented using [tf.keras](https://www.tensorflow.org/guide/keras/sequential_model) and [eager execution](https://www.tensorflow.org/guide/eager). The following is the sample output when the model in this tutorial trained for 30 epochs, and started with the prompt "Q":

<pre>
QUEENE:
I had thought thou hadst a Roman; for the oracle,
Thus by All bids the man against the word,
Which are so weak of care, by old care done;
Your children were in your holy love,
And the precipitation through the bleeding throne.

BISHOP OF ELY:
Marry, and will, my lord, to weep in such a one were prettiest;
Yet now I was adopted heir
Of the world's lamentable day,
To watch the next way with his father with his face?

ESCALUS:
The cause why then we are all resolved more sons.

VOLUMNIA:
O, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, it is no sin it should be dead,
And love and pale as any will to that word.

QUEEN ELIZABETH:
But how long have I heard the soul for this world,
And show his hands of life be proved to stand.

PETRUCHIO:
I say he look'd on, if I must be content
To stay him from the fatal of our country's bliss.
His lordship pluck'd from this sentence then for prey,
And then let us twain, being the moon,
were she such a case as fills m
</pre>

While some of the sentences are grammatical, most do not make sense. The model has not learned the meaning of words, but consider:

* The model is character-based. When training started, the model did not know how to spell an English word, or that words were even a unit of text.

* The structure of the output resembles a play—blocks of text generally begin with a speaker name, in all capital letters similar to the dataset.

* As demonstrated below, the model is trained on small batches of text (100 characters each), and is still able to generate a longer sequence of text with coherent structure.

## Setup

### Import TensorFlow and other libraries

In [2]:
import tensorflow as tf

import numpy as np
import os
import time

### Read the data

First, look in the text:

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
path_to_file = f'./drive/MyDrive/Tensorflow/DeepLearning.AI/C3/W4/Ungraded/Lab3/shakespeare.txt'

In [13]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print(f'Length of text: {len(text)} characters')

Length of text: 1115394 characters


In [14]:
# Take a look at the first 250 characters in text
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [15]:
# The unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

65 unique characters


## Process the text

### Vectorize the text

Before training, you need to convert the strings to a numerical representation.

The `tf.keras.layers.StringLookup` layer can convert each character into a numeric ID. It just needs the text to be split into tokens first.

In [53]:
example_texts = ['abcdefg', 'xyz']

chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

Now create the `tf.keras.layers.StringLookup` layer:

In [54]:
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None)

It converts from tokens to character IDs:

In [55]:
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[40, 41, 42, 43, 44, 45, 46], [63, 64, 65]]>

Since the goal of this tutorial is to generate text, it will also be important to invert this representation and recover human-readable strings from it. For this you can use `tf.keras.layers.StringLookup(..., invert=True)`.  

Note: Here instead of passing the original vocabulary generated with `sorted(set(text))` use the `get_vocabulary()` method of the `tf.keras.layers.StringLookup` layer so that the `[UNK]` tokens is set the same way.

In [56]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

This layer recovers the characters from the vectors of IDs, and returns them as a `tf.RaggedTensor` of characters:

In [57]:
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

You can `tf.strings.reduce_join` to join the characters back into strings.

In [58]:
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

In [59]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

### The prediction task

Given a character, or a sequence of characters, what is the most probable next character? This is the task you're training the model to perform. The input to the model will be a sequence of characters, and you train the model to predict the output—the following character at each time step.

Since RNNs maintain an internal state that depends on the previously seen elements, given all the characters computed until this moment, what is the next character?


### Create training examples and targets

Next divide the text into example sequences. Each input sequence will contain `seq_length` characters from the text.

For each input sequence, the corresponding targets contain the same length of text, except shifted one character to the right.

So break the text into chunks of `seq_length+1`. For example, say `seq_length` is 4 and our text is "Hello". The input sequence would be "Hell", and the target sequence "ello".

To do this first use the `tf.data.Dataset.from_tensor_slices` function to convert the text vector into a stream of character indices.

In [60]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(1006,), dtype=int64, numpy=array([31, 28, 26, ..., 44, 53,  2])>

In [61]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [62]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

R
O
M
E
O
:


N
a
y


In [63]:
seq_length = 100

The `batch` method lets you easily convert these individual characters to sequences of the desired size.

In [64]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'R' b'O' b'M' b'E' b'O' b':' b'\n' b'N' b'a' b'y' b',' b' ' b'm' b'a'
 b'r' b'k' b' ' b'y' b'o' b'u' b' ' b't' b'h' b'e' b'e' b'.' b'\n' b'\n'
 b'C' b'O' b'R' b'I' b'O' b'L' b'A' b'N' b'U' b'S' b':' b'\n' b'M' b'y'
 b' ' b'w' b'o' b'r' b'd' b',' b' ' b'y' b'o' b'u' b' ' b'h' b'a' b'v'
 b'e' b' ' b's' b'a' b'i' b'd' b' ' b'f' b'o' b'r' b',' b' ' b'a' b'n'
 b'd' b' ' b'y' b'o' b'u' b' ' b'm' b'u' b's' b't' b' ' b'a' b'c' b'q'
 b'u' b'a' b'i' b'n' b't' b'\n' b'T' b'o' b' ' b's' b'a' b'v' b'e' b' '
 b'y' b'o' b'u'], shape=(101,), dtype=string)


It's easier to see what this is doing if you join the tokens back into strings:

In [65]:
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())

b'ROMEO:\nNay, mark you thee.\n\nCORIOLANUS:\nMy word, you have said for, and you must acquaint\nTo save you'
b"r life, young shall be sometible\nOne day' dead, take I first beloved,\nWhere now, when fairing in Bloo"
b"d! Both, yea,\nTell me, what you do, if you change this way, let's\nBefore I stood to flatter them.\n\nMI"
b'RAND:\nMay I say again; but you say you out of all\nThe after of Perisly: know you offer out that you s'
b'tay?\n\nROTH:\nMy coffink from London and God forboabing,\nThat were as fatherity, adieute of I\ndail not '


For training you'll need a dataset of `(input, label)` pairs. Where `input` and
`label` are sequences. At each time step the input is the current character and the label is the next character.

Here's a function that takes a sequence as input, duplicates, and shifts it to align the input and label for each timestep:

In [66]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [67]:
split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [68]:
dataset_split = sequences.map(split_input_target)

In [69]:
for input_example, target_example in dataset_split.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'ROMEO:\nNay, mark you thee.\n\nCORIOLANUS:\nMy word, you have said for, and you must acquaint\nTo save yo'
Target: b'OMEO:\nNay, mark you thee.\n\nCORIOLANUS:\nMy word, you have said for, and you must acquaint\nTo save you'


### Create training batches

You used `tf.data` to split the text into manageable sequences. But before feeding this data into the model, you need to shuffle the data and pack it into batches.

In [70]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset_split
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .cache()
    .prefetch(tf.data.AUTOTUNE))

dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

## Build The Model

This section defines the model as a `keras.Model` subclass (For details see [Making new Layers and Models via subclassing](https://www.tensorflow.org/guide/keras/custom_layers_and_models)).

This model has three layers:

* `tf.keras.layers.Embedding`: The input layer. A trainable lookup table that will map each character-ID to a vector with `embedding_dim` dimensions;
* `tf.keras.layers.GRU`: A type of RNN with size `units=rnn_units` (You can also use an LSTM layer here.)
* `tf.keras.layers.Dense`: The output layer, with `vocab_size` outputs. It outputs one logit for each character in the vocabulary. These are the log-likelihood of each character according to the model.

In [34]:
# Length of the vocabulary in StringLookup Layer
vocab_size = len(ids_from_chars.get_vocabulary())

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [35]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim),
    tf.keras.layers.GRU(rnn_units, return_sequences=True),
    tf.keras.layers.Dense(vocab_size)
])

For each character the model looks up the embedding, runs the GRU one timestep with the embedding as input, and applies the dense layer to generate logits predicting the log-likelihood of the next character:

![A drawing of the data passing through the model](./img/text_generation_training.jpg)

Note: For training you could use a `keras.Sequential` model here. To  generate text later you'll need to manage the RNN's internal state. It's simpler to include the state input and output options upfront, than it is to rearrange the model architecture later. For more details see the [Keras RNN guide](https://www.tensorflow.org/guide/keras/rnn#rnn_state_reuse).

## Try the model

Now run the model to see that it behaves as expected.

First check the shape of the output:

In [36]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 66) # (batch_size, sequence_length, vocab_size)


In the above example the sequence length of the input is `100` but the model can be run on inputs of any length:

In [37]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ (64, 100, 256)              │          16,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru (GRU)                            │ (64, 100, 1024)             │       3,938,304 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (64, 100, 66)               │          67,650 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 4,022,850 (15.35 MB)

 Trainable params: 4,022,850 (15.35 MB)

 Non-trainable params: 0 (0.00 B)

To get actual predictions from the model you need to sample from the output distribution, to get actual character indices. This distribution is defined by the logits over the character vocabulary.

Note: It is important to _sample_ from this distribution as taking the _argmax_ of the distribution can easily get the model stuck in a loop.

Try it for the first example in the batch:

In [38]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

This gives us, at each timestep, a prediction of the next character index:

In [39]:
sampled_indices

array([37,  5, 62, 22, 16, 49, 21, 23, 40, 11, 33, 63, 11, 30,  5, 56, 28,
       54, 41, 61, 42,  7, 29, 53, 26, 13, 47,  8, 47, 64, 57, 64, 42, 30,
       55, 26, 47,  7, 22, 38, 60,  7, 63, 31,  6, 38, 36, 24, 17, 18, 57,
       23, 50, 27, 40, 20, 45,  4, 41, 32, 41, 60, 43,  2, 63, 53, 65, 57,
       24, 40, 64,  6, 10, 52, 33,  6, 59, 45, 59, 36, 56, 42,  3, 50, 52,
       18, 39,  0, 25, 17, 18, 13, 34, 52, 46, 37, 54, 24,  8, 20])

Decode these to see the text predicted by this untrained model:

In [40]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b'button, a duellist, a duellist; a gentleman of the\nvery first house, of the first and second cause:\n'

Next Char Predictions:
 b"X&wICjHJa:Tx:Q&qOobvc,PnM?h-hyrycQpMh,IYu,xR'YWKDErJkNaGf$bSbud xnzrKay'3mT'tftWqc!kmEZ[UNK]LDE?UmgXoK-G"


## Train the model

At this point the problem can be treated as a standard classification problem. Given the previous RNN state, and the input this time step, predict the class of the next character.

### Attach an optimizer, and a loss function

The standard `tf.keras.losses.sparse_categorical_crossentropy` loss function works in this case because it is applied across the last dimension of the predictions.

Because your model returns logits, you need to set the `from_logits` flag.


In [41]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [42]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 66)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.190402, shape=(), dtype=float32)


A newly initialized model shouldn't be too sure of itself, the output logits should all have similar magnitudes. To confirm this you can check that the exponential of the mean loss is approximately equal to the vocabulary size. A much higher loss means the model is sure of its wrong answers, and is badly initialized:

In [43]:
tf.exp(example_batch_mean_loss).numpy()

66.04934

Configure the training procedure using the `tf.keras.Model.compile` method. Use `tf.keras.optimizers.Adam` with default arguments and the loss function.

In [44]:
model.compile(optimizer='adam', loss=loss, metrics=['sparse_categorical_accuracy'])

### Execute the training

To keep training time reasonable, use 20 epochs to train the model. In Colab, set the runtime to GPU for faster training.

In [45]:
EPOCHS = 20

In [46]:
history = model.fit(dataset, epochs=EPOCHS)

Epoch 1/20
172/172 ━━━━━━━━━━━━━━━━━━━━ 13s 53ms/step - loss: 3.0741 - sparse_categorical_accuracy: 0.2373
Epoch 2/20
172/172 ━━━━━━━━━━━━━━━━━━━━ 9s 55ms/step - loss: 1.9467 - sparse_categorical_accuracy: 0.4279
Epoch 3/20
172/172 ━━━━━━━━━━━━━━━━━━━━ 10s 57ms/step - loss: 1.6585 - sparse_categorical_accuracy: 0.5069
Epoch 4/20
172/172 ━━━━━━━━━━━━━━━━━━━━ 9s 55ms/step - loss: 1.5069 - sparse_categorical_accuracy: 0.5452
Epoch 5/20
172/172 ━━━━━━━━━━━━━━━━━━━━ 10s 57ms/step - loss: 1.4169 - sparse_categorical_accuracy: 0.5680
Epoch 6/20
172/172 ━━━━━━━━━━━━━━━━━━━━ 10s 58ms/step - loss: 1.3544 - sparse_categorical_accuracy: 0.5840
Epoch 7/20
172/172 ━━━━━━━━━━━━━━━━━━━━ 10s 59ms/step - loss: 1.3043 - sparse_categorical_accuracy: 0.5968
Epoch 8/20
172/172 ━━━━━━━━━━━━━━━━━━━━ 11s 61ms/step - loss: 1.2613 - sparse_categorical_accuracy: 0.6079
Epoch 9/20
172/172 ━━━━━━━━━━━━━━━━━━━━ 21s 61ms/step - loss: 1.2237 - sparse_categorical_accuracy: 0.6183
Epoch 10/20
172/172 ━━━━━━━━━━━━━━━━━━━

## Generate text

The simplest way to generate text with this model is to run it in a loop, and keep track of the model's internal state as you execute it.

![To generate text the model's output is fed back to the input](./img/text_generation_sampling.jpg)

Each time you call the model you pass in some text and an internal state. The model returns a prediction for the next character and its new state. Pass the prediction and state back in to continue generating text.


The following makes a single step prediction:

In [47]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Embedding layer
    x = self.model.layers[0](input_ids)
    # GRU layer
    x = self.model.layers[1](x, initial_state=states)
    # Get the hidden state of the last timestep
    states = x[:, -1, :]
    # Dense layer
    predicted_logits = self.model.layers[2](x)

    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature

    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [48]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

Run it in a loop to generate some text. Looking at the generated text, you'll see the model knows when to capitalize, make paragraphs and imitates a Shakespeare-like writing vocabulary. With the small number of training epochs, it has not yet learned to form coherent sentences.

In [49]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

ROMEO:
Good Clarence, because;
Even atto his body, and not again;
I'll join'd my name being it are veins, our beasuable.

Third Citizen:
Ohe way, O surely, I thank you; having him,
My father and sobubjed ona,
Who from my brother's sake, for what I wear
The ear impeecius passon'd knave,
In Jove's son I, bear the sweetest love;
Ere fares your walls and me, to flowere the subject.

MERBY:
Sound dream of thoseth not.

First Citizen:
Ye'er 'Buttifal, let's torme. An honour and Richard!

DUKE OF YORK:
Tut, though I want my bog-play'd
To the use-peeceived where I.

QUEEN MARGARET:
Stealth, I know, they say. I'll tell you hear my necess?

KING RICHARD III:
A lamanishment of holy men,
And he my sweets, ere he over all the haughty
you are you, sir! hark?

First Lord:
Behold, fear'st me, for speaking, see you once.

HENRY BOLINGBROKE:
Who knows not the king?

Nurse:
May not you be-a--
That both Disposion will go fear, you hereford's rage
With our closk--ither our accused;

ISALEO:
Granday!

MENEN

The easiest thing you can do to improve the results is to train it for longer (try `EPOCHS = 30`).

You can also experiment with a different start string, try adding another RNN layer to improve the model's accuracy, or adjust the temperature parameter to generate more or less random predictions.

If you want the model to generate text *faster* the easiest thing you can do is batch the text generation. In the example below the model generates 5 outputs in about the same time it took to generate 1 above.

In [50]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

results = tf.strings.join(result)
end = time.time()

for text in results:
    print(text.numpy().decode('utf-8'), '\n\n' + '_'*80)

print('\nRun time:', end - start)

ROMEO:
I saw no only inevion, see's heaven holds it.

LUCIO:
Ay, Catesbay, whom I do hold you throwing on?

PETRUCHIO:
A parading than in true afford against
A lady under Caiusa?'

HERMIONE:
Ay,
You would say we all brains with it.

DUKE VINCENTIO:
It is not yet I know, where's Madianca:
Farewell! one east, ere I loved me well, nor
thorneyses out. Come only
The Earl of God, for seal'd Miphation,
It strain with Miratation, love--
Whom thou wouldst have thee in our father,
Thou strokest up that senators To, Pomp,
My prayers will begets it to thy royal tear.
It love thee, how thou art no letters;
Some free your father shall be blood, call'd by
Ere we wife fond; for if thou slower
Too learning. You, worthy Margary,
Patience thou art.
Come, clouds, or else thy kingdom she is ago;
And not you to your oaths; the tames
Whom art as what death?

Clown:
Alas, poor young me, sir; as they stay to us.

PROSPERO:
No, no.

VOLUMNIA:
Ho! You two, an heigh; If you are never
With the unacquainted women o

## Export the generator

This single-step model can easily be [saved and restored](https://www.tensorflow.org/guide/saved_model), allowing you to use it anywhere a `tf.saved_model` is accepted.

In [51]:
tf.saved_model.save(one_step_model, 'one_step')
one_step_reloaded = tf.saved_model.load('one_step')

In [52]:
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(100):
  next_char, states = one_step_reloaded.generate_one_step(next_char, states=states)
  result.append(next_char)

print(tf.strings.join(result)[0].numpy().decode("utf-8"))

ROMEO:
Come to that verse, you have been alive truly the king's snaxcubsol, new
Justerids our lawful ade,-
